In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support import expected_conditions as E
from selenium.webdriver import ActionChains
import time
import openpyxl

#start chrome
options = webdriver.ChromeOptions() 
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=options, executable_path=r'c:\Users\Chan Kin Yan\Desktop\ZOE\BUSI\ISOM\ISOM3400\project\chromedriver.exe')
driver.maximize_window()

driver.get("https://www.openrice.com/en/hongkong/restaurants")
url_main = 'https://www.openrice.com'
url_district = ['Yuen Long','Tsuen Wan','Causeway Bay','Mong Kok','Central','Tsim Sha Tsui']
jap = 0

#creat an excel file
path = 'restaurant_MOREinfo_6Districts.xlsx'
wb = openpyxl.Workbook(); 
sheet = wb.active; sheet.title = "restaurant_info"
sheet.append(['name', 'price', 'bookmark', 'happy', 'sad', 'food_type','location',
                'num_branch', 'num_seat', 'pay_list', 'other_facility',
                'taste','decor','service','hygiene','value','overall'])
wb.save(path)

#function for sending data to the created workbook
def excel(info_list):
    wb = openpyxl.load_workbook(path)
    sheet = wb.active
    for i in info_list:
        sheet.append(i)
    wb.save(path)
    print("open and save info")

#check duplication
def duplicate(name_list, name):
    if name in name_list:
        return True
    else: 
        name_list.append(name)
        return False

def get_num(txt):
    return [s for s in txt if s.isdigit()]

# get more info by clicking into the individual restaurant
def get_more_info(bs_more):
    time.sleep(2)
    #get rating
    try:
        rate_list = bs_more.find_all ('div',{'class':'header-score-details-right-item'})
        criteria = { }; total = 0
        for rate_item in rate_list:
            c_name = str(rate_item.find('div',{'class':'header-score-details-right-item-title'}).get_text()) 
            rate = rate_item.find_all('div')[1].attrs['class'][2]
            rate = int(get_num(rate)[0])
            total+=rate
            criteria[c_name]=rate
        criteria['overall']=total
    except:  return None
        
    # get number of branches
    try:
        num_branch=bs_more.find('section',class_='all-branches-section').a.get_text()
        num_branch=int(''.join(get_num(num_branch)))
    except:
        num_branch = 1

    rest_info = bs_more.find('div',class_='left-col').find('div',class_='or-section-group pois-filter-feature')
    # get payment method
    pay_list = []
    try:
        payment = rest_info.find('div',class_="comma-tags").find_all('span')
        for pay in payment:
            pay_list.append(str(pay.get_text()))
        pay_list=','.join(pay_list)
    except: pass

    #get num_seat
    try:
        num_seat = str(rest_info.find('div',class_='content'))
        num_seat=int(''.join(get_num(num_seat)))
    except:  return None

    # other facility
    try:
        rest_info = bs_more.find('div',class_='left-col').find('div',class_='or-section-group pois-filter-feature')
        other_info = rest_info.find_all('span',class_='condition-name')
        other_facility = []
        for item in other_info:
            other_facility+=item.contents
        other_facility = ','.join(other_facility)
    except: return None
    '''
    #satisfaction level
        try:
            satisfaction_list = bs_more.find_all('div',class_='score-div')
            satisfaction = {}
            score_cate = ['happy','ok','sad']
            for i in range(3):
                satisfaction[score_cate[i]] = satisfaction_list[i].get_text()
        except: return None
    '''
    return list(criteria.values()), num_branch, pay_list, num_seat, other_facility, 

#get info
def get_info(bs, district, info_list,url):
    global name_list
    item_list =  bs.find_all('section',class_='content-wrapper')
    info_jap=[]

    #get info of each restaurant
    for item in item_list:

        name = item.find('h2').a.string
        #check duplication
        if duplicate(name_list,name): continue
        print(name)

        try:
            price = item.find('div',{'class':'icon-info icon-info-food-price'}).span.string
        except AttributeError:
            continue
        
        try:
            bookmark = item.find('div',{'class':'text bookmarkedUserCount js-bookmark-count'}).get("data-count")
        except AttributeError:
            continue

        try:
            sad = item.find('span',{'class':'score highlight'}).string
        except AttributeError:
            continue
        
        try:
            happy = item.find('span',{'class':'score score-big highlight'}).string
        except AttributeError:
            continue
            
        try:
            food_type = item.find('li').string
        except AttributeError:
            continue

        try:
            location = item.find('div',{'class':'icon-info address'}).a.string
            if location != district: continue
        except AttributeError:
            continue

        #save data
        one_res = [name, price, bookmark, happy, sad, food_type, location]
        if 'Japan' in food_type:
            global jap; jap+=1
            info_jap.append(one_res)
        else:
            info_list.append(one_res); time.sleep(2)
     
    #enter new page for extra info if janpanese food
    for i in range(len(info_jap)):
        name_japan = info_jap[i][0]
        try:
            # click into the individual restaurant
            while True:
                try:
                    moreinfo = W(driver,15).until(E.presence_of_element_located((By.PARTIAL_LINK_TEXT,str(name_japan))))
                    moreinfo.click()
                    break
                except:
                    driver.get(url); time.sleep(5)
                    #driver.execute_script("window.history.go(-1)"); time.sleep(5)

            #getting more info of the restaurant    
            time.sleep(2)
            bs_more=BeautifulSoup(driver.page_source, 'html.parser')
            bs_more.prettify()
            
            try:
                criteria, num_branch, pay_list, num_seat, other_facility = get_more_info(bs_more)
                
                print('getting more')
            except: time.sleep(3) ;continue
                
            print('enter',name_japan)
            # go back to the main page
            try:
                driver.execute_script("window.history.go(-1)")
                driver.implicitly_wait(1)
                time.sleep(5)
                print('go_out')
            except:
                print('go back failed')
                driver.quit()
        except:
            print('cannot',name_japan); continue

        #save the data 
        info_jap[i]+= [num_branch, num_seat, pay_list, other_facility]
        for j in criteria:
            info_jap[i].append(j)
    #sava data to main info list
    info_list+=info_jap
    time.sleep(2)


#main
cookie_pressed = False
name_list = []

for district in url_district:
    info_list = []
    #find search box
    try:
        search_box = driver.find_element_by_name("where")
        search_box.clear() 
        search_box.send_keys(district)
        search = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[4]/div/button').click()
        time.sleep(4)
    except:
        print('error'); driver.quit()

    #ready to get info
    endsearch = False; page = 1; url= driver.current_url
    
    while not endsearch:
        info_list = [] 
        #cookies
        try:
            if (page == 1 and not cookie_pressed ):
                print('cookie pressing')
                cookie = W(driver, 5).until(E.presence_of_element_located((By.XPATH,'//*[@id="cookies-agreement"]/div/button')))
                type(cookie)
                cookie.click()
                cookie_pressed = True
        except:
            print("no cookies")

        time.sleep(5)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        bs.prettify()
        # get infomation in this page
        get_info(bs,district,info_list,url)

        #save data in this page
        excel(info_list)

        #go to next page
        #find the link of next page 
        time.sleep(3)
        next_page = bs.find('section', {'class':'js-pois-pagination pull-right'}).find('a',class_='pagination-button next js-next')      
        if next_page == None:
            endsearch = True
        else:
            next_page = next_page.attrs['href']
            url=url_main+str(next_page)
            driver.get(url)
            time.sleep(2)
            
    print('done',district)

driver.quit()
print(jap) 